In [2]:
!pip install lambeq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.3/364.3 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 13.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.w

In [3]:
!pip install pytket-qiskit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.6/308.6 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.6/249.6 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.7/323.7 kB 33.7 MB/s eta 0:00:00
  Installing build depende

In [16]:
pip install "discopy>=1.1.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 3.2 MB/s eta 0:00:00


In [45]:
from lambeq import BobcatParser, Rewriter, IQPAnsatz, AtomicType
from pytket.circuit import Circuit, Qubit
from pytket.extensions.qiskit import AerStateBackend
from pytket.passes import DecomposeBoxes
import numpy as np

parser = BobcatParser()
sentence1 = "Dog sat on the floor."
sentence2 = "The cat sat on the mat."

diagram1 = parser.sentence2diagram(sentence1)
diagram2 = parser.sentence2diagram(sentence2)

rewriter = Rewriter(['prepositional_phrase', 'determiner'])
circuit1 = rewriter(diagram1)
circuit2 = rewriter(diagram2)

N = AtomicType.NOUN
S = AtomicType.SENTENCE
ob_map = {N: 2, S: 1}

ansatz = IQPAnsatz(ob_map=ob_map, n_layers=1)
quantum_circuit1 = ansatz(circuit1)
quantum_circuit2 = ansatz(circuit2)

def convert_to_tket_circuit(lambeq_circuit):
    tket_circuit = Circuit()
    qubits = {}

    for layer in lambeq_circuit.layers:
        box = layer.box
        for q in range(len(box.dom)):
            if q not in qubits:
                tket_circuit.add_qubit(Qubit(q))
                qubits[q] = Qubit(q)
        if box.name == 'Rx':
            tket_circuit.Rx(box.data[0], qubits[0])
        elif box.name == 'Ry':
            tket_circuit.Ry(box.data[0], qubits[0])
        elif box.name == 'Rz':
            tket_circuit.Rz(box.data[0], qubits[0])
        elif box.name == 'CX':
            tket_circuit.CX(qubits[0], qubits[1])
        elif box.name == 'H':
            tket_circuit.H(qubits[0])
    DecomposeBoxes().apply(tket_circuit)
    return tket_circuit

tket_circuit1 = convert_to_tket_circuit(quantum_circuit1)
tket_circuit2 = convert_to_tket_circuit(quantum_circuit2)

backend = AerStateBackend()
state1 = backend.run_circuit(tket_circuit1).get_state()
state2 = backend.run_circuit(tket_circuit2).get_state()

similarity = np.abs(np.dot(state1.conjugate(), state2))**2

print(f"Similarity between the sentences: {similarity}")

Similarity between the sentences: 0.9999999999999996
